## Descrição da Base
-   Base de dados de de Tuberculose;
-   Dados coletados através do TBWEB em SÃO PAULO;
-   Registros coletados de 2006 a 2016;
-   Amostra: 46 atributos e 103846 registros;

### Processo de Execução
   - Seleção de variáveis  
   - Tratamento e tabulação de variáveis
   - Aplicação de Modelos de Correlação
   - Execução dos Modelos de Clusterização
   - Apicação do modelo de Regressão Logistica

In [25]:
#__Bibliotecas
import pandas as pd

import numpy as np   
import seaborn as sns   
import matplotlib.pyplot as plt
import seaborn as sns

from pandas_profiling import ProfileReport
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
import pingouin as pg


#Definir padrão aleatório utilizado no random_state
np.random.seed(13)
#__Modelos


In [26]:
# Precisão dos dados em duas casas decimais
%precision %.2f
pd.options.display.float_format = '{:,.5f}'.format

In [27]:
#__Conexão com o dataset em CSV
# df= pd.read_csv('tuberculosis_data.csv', sep=';')
df= pd.read_csv('../Dataset/tuberculosis_data.csv', sep=';')

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103846 entries, 0 to 103845
Data columns (total 47 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          103846 non-null  int64  
 1   racaCor             89811 non-null   object 
 2   faixaEtaria         103846 non-null  object 
 3   sexo                103846 non-null  object 
 4   gestante            34483 non-null   object 
 5   ESCOLARID           83686 non-null   object 
 6   TIPOCUP             91755 non-null   object 
 7   codTratAnt          100217 non-null  float64
 8   tratouha            103846 non-null  float64
 9   sitAtual            103846 non-null  object 
 10  tipoCaso            103846 non-null  object 
 11  FORMACLIN1          103838 non-null  object 
 12  FORMACLIN2          3802 non-null    object 
 13  FORMACLIN3          231 non-null     object 
 14  classif             103845 non-null  object 
 15  descoberta          102239 non-nul

In [29]:
# df_select = df[['racaCor', 'sexo',
#                 'ESCOLARID', 'TIPOCUP', 'sitAtual', 
#                 'tipoCaso', 'FORMACLIN1', 'classif',
#                 'aids', 'DIABETES', 'ALCOOLISMO', 
#                 'MENTAL', 'DROGADICAO', 'TABAGISMO', 
#                 'idade', 'esqIni', 'motMudEsquema', 'mdEsquema']]

# df_select

df_select = df[['cultEsc', 'testesensibilidade', 'classif', 'FORMACLIN1', 
                'bac', 'DROGADICAO', 'hiv', 'CULTOUTRO', 'aids', 'ALCOOLISMO',
                'nDosesSeg', 'instTrat', 'TOTCOMUNIC', 'esqIni', 'nDosesPri', 
                'faixaEtaria', 'COMUNICEXA', 'HISTOPATOL', 'tipoCaso', 'TABAGISMO',
                'TIPOCUP', 'sexo', 'descoberta', 'mtvInter1', 'sitAtual', 'idade', 'RX']]

df_select

,cultEsc,testesensibilidade,classif,FORMACLIN1,bac,DROGADICAO,hiv,CULTOUTRO,aids,ALCOOLISMO,...,HISTOPATOL,tipoCaso,TABAGISMO,TIPOCUP,sexo,descoberta,mtvInter1,sitAtual,idade,RX
0,Neg,NaN,P+E,Pul,Neg,N,Pos,Neg,S,N,...,N/realiz,Novo,N,Outra,M,Elucidacao Diagn. em Internacao,Elucidacao Diagnostica,Abandono,26.00000,Susp TB
1,N/realiz,NaN,Pul,Pul,Pos,S,Neg,N/realiz,N,S,...,N/realiz,Novo,N,Outra,M,Demanda Ambulatorial,Insuficiencia Respiratoria Aguda,Cura,38.00000,Susp TB
2,Pos,NaN,Pul,Pul,Pos,N,N/realiz,N/realiz,N,N,...,N/realiz,Retr Aband,N,Outra,M,Busca Ativa na Comunidade,NaN,Abandono,24.00000,N/realiz
3,NaN,S,P+E,Pul,NaN,N,Pos,Pos,S,N,...,N/realiz,Novo,N,Dona de Casa,F,Elucidacao Diagn. em Internacao,Abscesso,Cura,35.00000,Susp TB
4,N/realiz,NaN,Pul,Pul,Pos,N,Neg,N/realiz,N,S,...,N/realiz,Novo,N,Outra,M,Elucidacao Diagn. em Internacao,Elucidacao Diagnostica,Cura,41.00000,Susp TB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103841,Pos,S,Pul,Pul,Pos,S,Neg,N/realiz,N,N,...,N/realiz,Novo,S,Desempregado,F,Investigacao de Contatos,NaN,Cura,28.00000,Susp c/cavid
103842,N/realiz,NaN,Pul,Pul,Neg,N,Neg,N/realiz,N,N,...,N/realiz,Novo,N,Outra,M,Urgencia / Emergencia,NaN,Cura,36.00000,Susp TB
103843,N/realiz,N,Ext,Ganglionar Periferica,N/realiz,N,Pos,NaN,S,N,...,NaN,Novo,N,Outra,F,Demanda Ambulatorial,NaN,Abandono,31.00000,Normal
103844,N/realiz,N,Ext,Intestinal,Neg,N,N/realiz,Neg,N,N,...,N/realiz,Novo,N,Aposentado,F,Demanda Ambulatorial,NaN,Cura,66.00000,Normal


In [30]:
def tipopaciente(motMudEsquema, mdEsquema):
    if motMudEsquema == 'Intolerancia/Toxicidade' and mdEsquema == 'S':
        return 1
    elif motMudEsquema == 'Resistencia Medicamentosa' and mdEsquema == 'S':
        return 1
    elif motMudEsquema == 'Outro Motivo' and mdEsquema == 'S':
        return 1
    else:
        return 0
    
def tiporesistencia(resistencia):
    if resistencia == 'AND':
        return 1
    elif resistencia == 'SENS':
        return 1
    elif resistencia == 'TB MR':
        return 1
    elif resistencia == 'TB R':
        return 1
    else:
        return 0

In [31]:
#__ Tranformação
# Cria coluna de Status do Paciente
df['Status_Paciente'] = df.apply(lambda row: tipopaciente(row['motMudEsquema'], row['mdEsquema']), axis=1)

# Cria coluna de Status Resistencia
df['Status_Resistencia'] = df.apply(lambda row: tiporesistencia(row['resistencia']), axis=1)

# Categoriza a coluna de idade
df['idade'] = df['idade'].astype(int)

# Categoriza a coluna de idade
pd.qcut(df.idade, q=4)

# Substitui os intervalos gerados na coluna idade
df['idade'] = pd.qcut(df.idade, q=4, labels=['0-22', '23-39', '40-54', 'Mais de 54']).astype(object)
display(df)


,Unnamed: 0,racaCor,faixaEtaria,sexo,gestante,ESCOLARID,TIPOCUP,codTratAnt,tratouha,sitAtual,...,tipoTrat,nDosesPri,nDosesSeg,idade,HISTOPATOL,OUTRAIMUNO,resistencia,mtvInter1,Status_Paciente,Status_Resistencia
0,2,Pardo,20_29,M,NaN,De 1 a 3 anos,Outra,1.00000,0.00000,Abandono,...,Auto-Administrado,0.00000,0.00000,23-39,N/realiz,N,NaN,Elucidacao Diagnostica,0,0
1,4,Pardo,30_39,M,NaN,De 4 a 7 anos,Outra,1.00000,0.00000,Cura,...,Supervisionado,0.00000,0.00000,40-54,N/realiz,N,NaN,Insuficiencia Respiratoria Aguda,0,0
2,7,NaN,20_29,M,NaN,De 4 a 7 anos,Outra,NaN,0.00000,Abandono,...,Supervisionado,0.00000,0.00000,0-22,N/realiz,N,SENS,NaN,0,1
3,15,NaN,30_39,F,NaN,NaN,Dona de Casa,1.00000,0.00000,Cura,...,Auto-Administrado,0.00000,0.00000,23-39,N/realiz,N,SENS,Abscesso,0,1
4,45,Branco,40_49,M,NaN,De 4 a 7 anos,Outra,1.00000,0.00000,Cura,...,NaN,0.00000,0.00000,40-54,N/realiz,N,NaN,Elucidacao Diagnostica,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103841,208610,Preto,20_29,F,N,De 4 a 7 anos,Desempregado,1.00000,0.00000,Cura,...,Supervisionado,32.00000,61.00000,23-39,N/realiz,N,NaN,NaN,0,0
103842,208615,Branco,30_39,M,NaN,De 1 a 3 anos,Outra,1.00000,0.00000,Cura,...,Supervisionado,60.00000,54.00000,23-39,N/realiz,N,NaN,NaN,0,0
103843,208616,Preto,30_39,F,N,De 8 a 11 anos,Outra,1.00000,0.00000,Abandono,...,Supervisionado,31.00000,0.00000,23-39,NaN,N,NaN,NaN,0,0
103844,208617,Pardo,60_69,F,N,De 12 a 14 anos,Aposentado,1.00000,0.00000,Cura,...,Supervisionado,40.00000,82.00000,Mais de 54,N/realiz,S,NaN,NaN,0,0


In [32]:
# Analisa os dados utlizando o pandas profiling
report = ProfileReport(df_select, minimal=True, title='Hansys Project')
# Salva o arquivo html com as análises realizadas pelo pandas profiling
report.to_file("profileTB.html")

C:\Users\albert.DESKTOP-015ON1O\AppData\Roaming\Python\Python39\site-packages\pandas_profiling\utils\dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [33]:
# #__ Balanceamento da Base - Método random_state
# Filtra os registros onde o 'Status_Paciente' é igual a 1 e igual a )
class1 = df[df['Status_Resistencia'] == 1]
class2 = df[df['Status_Resistencia'] == 0]


# Determina o tamanho mínimo entre o número de registros da classe 1 (com Resistência) e da classe 2 (Sem Resistência)
min_class_size = min(class1.shape[0], class2.shape[0])

# Seleciona uma amostra aleatória de registros da classe 1 com tamanho igual a min_class_size (Com Resistência)
subset_classe1 = class1.sample(n=min_class_size, random_state=1)

# Seleciona uma amostra aleatória de registros da classe 2 com tamanho igual a min_class_size (Sem Resistência)
subset_classe2 = class2.sample(n=min_class_size, random_state=1)

# Concatena os dois subconjuntos balanceados em um único DataFrame
subset_balanceado = pd.concat([subset_classe1, subset_classe2])

# Embaralha aleatoriamente os registros no DataFrame subset_balanceado
df = subset_balanceado.sample(frac=1, random_state=1)


display(df)


,Unnamed: 0,racaCor,faixaEtaria,sexo,gestante,ESCOLARID,TIPOCUP,codTratAnt,tratouha,sitAtual,...,tipoTrat,nDosesPri,nDosesSeg,idade,HISTOPATOL,OUTRAIMUNO,resistencia,mtvInter1,Status_Paciente,Status_Resistencia
74351,160988,Branco,40_49,M,NaN,NaN,NaN,1.00000,0.00000,Cura,...,Supervisionado,0.00000,0.00000,40-54,N/realiz,N,SENS,Elucidacao Diagnostica,0,1
64532,144667,Branco,15_19,M,NaN,De 8 a 11 anos,Desempregado,1.00000,0.00000,Cura,...,Supervisionado,24.00000,96.00000,0-22,NaN,N,SENS,Insuficiencia Respiratoria Aguda,0,1
67919,150681,Branco,20_29,M,NaN,De 4 a 7 anos,Desempregado,1.00000,0.00000,Cura,...,Supervisionado,13.00000,39.00000,23-39,N/realiz,N,SENS,NaN,0,1
30194,96679,Preto,50_59,M,NaN,De 8 a 11 anos,Outra,1.00000,0.00000,Cura,...,Supervisionado,38.00000,80.00000,Mais de 54,N/realiz,N,SENS,NaN,0,1
41121,111344,Pardo,30_39,F,S,De 4 a 7 anos,Dona de Casa,1.00000,0.00000,Cura,...,Supervisionado,24.00000,48.00000,23-39,N/realiz,N,TB R,NaN,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48811,121479,Pardo,20_29,M,NaN,De 4 a 7 anos,Outra,1.00000,0.00000,Cura,...,Auto-Administrado,9.00000,45.00000,0-22,N/realiz,N,SENS,Insuficiencia Respiratoria Aguda,0,1
26937,92189,Pardo,20_29,M,NaN,De 8 a 11 anos,Outra,1.00000,0.00000,Cura,...,Supervisionado,28.00000,45.00000,23-39,N/realiz,N,NaN,NaN,0,0
58039,134572,Branco,70_79,M,NaN,De 8 a 11 anos,Aposentado,1.00000,0.00000,Cura,...,Supervisionado,29.00000,0.00000,Mais de 54,NaN,N,TB R,NaN,1,1
102168,205073,Pardo,10_14,F,N,De 4 a 7 anos,Outra,1.00000,0.00000,Cura,...,Supervisionado,22.00000,36.00000,0-22,N/realiz,N,SENS,NaN,0,1


In [34]:
df_select = df[['cultEsc', 'testesensibilidade', 'classif', 'FORMACLIN1', 
                'bac', 'DROGADICAO', 'hiv', 'CULTOUTRO', 'aids', 'ALCOOLISMO',
                'nDosesSeg', 'instTrat', 'TOTCOMUNIC', 'esqIni', 'nDosesPri', 
                'faixaEtaria', 'COMUNICEXA', 'HISTOPATOL', 'tipoCaso', 'TABAGISMO',
                'TIPOCUP', 'sexo', 'descoberta', 'mtvInter1', 'sitAtual', 'idade', 'RX', 'Status_Resistencia']]

df_select.head(3)

,cultEsc,testesensibilidade,classif,FORMACLIN1,bac,DROGADICAO,hiv,CULTOUTRO,aids,ALCOOLISMO,...,tipoCaso,TABAGISMO,TIPOCUP,sexo,descoberta,mtvInter1,sitAtual,idade,RX,Status_Resistencia
74351,N/realiz,NaN,Pul,Pul,Pos,N,Neg,N/realiz,N,N,...,Novo,N,NaN,M,Urgencia / Emergencia,Elucidacao Diagnostica,Cura,40-54,Susp TB,1
64532,N/realiz,S,Pul,Pul,Pos,N,Neg,N/realiz,N,N,...,Novo,N,Desempregado,M,Elucidacao Diagn. em Internacao,Insuficiencia Respiratoria Aguda,Cura,0-22,N/realiz,1
67919,Pos,NaN,Pul,Pul,Pos,S,Pos,N/realiz,N,N,...,Novo,S,Desempregado,M,Demanda Ambulatorial,NaN,Cura,23-39,N/realiz,1


In [35]:
# # Salvar o dataframe
df_select.to_csv('../Dataset/resultado_TB.csv', index=False)
# df_subset.to_csv('resultado_TB.csv', index=False)


* Qui-quadrado

In [36]:
# Calculo do qui-quadrado em relação à variável de status do paciente TB
# Cria um DataFrame vazio para armazenar os resultados do qui-quadrado
all_pvalue = pd.DataFrame(columns=['chi2', 'pval', 'dof', 'expected', 'Feature'])

# Pandas index com os nomes das colunas
feature_names = df.columns

# Realiza qui-quadrado para cada feature
for i in feature_names:
    # Cria um dataframe com duas colunas: feature i e Status Paciente
    df_chisquare = df.loc[:, [i, 'Status_Resistencia']].dropna()
    
    # Cria a tabela de contingência
    contingency_table = pd.crosstab(df_chisquare[i].values.flatten(), df_chisquare['Status_Resistencia'].values.flatten())
    
    # Realiza o teste qui-quadrado
    chi2, pval, dof, expected = chi2_contingency(contingency_table)
    
    # Cria um DataFrame com os resultados
    result_df = pd.DataFrame([[chi2, pval, dof, expected, i]], columns=['chi2', 'pval', 'dof', 'expected', 'Feature'])
    
    # Concatena o DataFrame ao DataFrame principal
    all_pvalue = pd.concat([all_pvalue, result_df], ignore_index=True)

# Filtra as features com valores de p < 0.05
significant_features = all_pvalue.query('pval <= 0.05').sort_values(by='chi2', ascending=False)

# Mostra as métricas para as features significantes
print(significant_features[['chi2', 'pval', 'dof', 'Feature']])


           chi2    pval  dof             Feature
48 53,224.00008 0.00000    1  Status_Resistencia
18  8,567.75400 0.00000    3             cultEsc
24  1,833.45900 0.00000    1  testesensibilidade
14  1,232.30604 0.00000    3             classif
11  1,226.75012 0.00000   13          FORMACLIN1
16    740.07900 0.00000    3                 bac
29    720.19731 0.00000    1          DROGADICAO
23    517.95680 0.00000    3                 hiv
19    483.11241 0.00000    3           CULTOUTRO
25    474.38638 0.00000    1                aids
27    460.02895 0.00000    1          ALCOOLISMO
41    456.75948 0.00000  246           nDosesSeg
34    447.36703 0.00000    6            instTrat
31    426.87585 0.00000   64          TOTCOMUNIC
35    412.46315 0.00000    3              esqIni
40    401.80543 0.00000  104           nDosesPri
2     391.89210 0.00000   11         faixaEtaria
32    364.56889 0.00000   60          COMUNICEXA
43    324.34495 0.00000    2          HISTOPATOL
10    322.28722 0.00